# Import the libraries that will be used throughout the examination of the data

In [1]:
# Importing important Libraries used by all
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Import library for split and train
from sklearn.model_selection import train_test_split

# Import for linear Regression
from sklearn.linear_model import LinearRegression

# Import for PCA
from sklearn.decomposition import PCA

# Import for Random Forest
from sklearn.ensemble import RandomForestRegressor

# Import for Decision Tree Reg
from sklearn.tree import DecisionTreeRegressor

# Import for K-Fold
from sklearn.svm import SVC

# K-Fold cross validation
from sklearn.model_selection import cross_val_score

# Import for normalizing
from sklearn.preprocessing import StandardScaler

# Import for finding the error of the model
from sklearn import metrics

# Import for Polynomial Regression
from sklearn.preprocessing import PolynomialFeatures

#Backward Elimination
import statsmodels.api as sm

#K-Fold Cross Validation
from sklearn.model_selection import cross_val_score

#Applying PCA
from sklearn.decomposition import PCA

# Load in the premier league data set and clean it up so it can be examined

In [2]:
# Loading in the data set being used
dataset = pd.read_csv('complete_gws.csv', encoding="ISO-8859-1")
dataset_test = pd.read_csv('complete_gws.csv', encoding="ISO-8859-1")

# Clean up the data
# DROP VALUES:
dataset.drop(["name", "kickoff_time", "kickoff_time_formatted"], axis=1, inplace=True)

dataset_test.drop(["name", "kickoff_time", "kickoff_time_formatted"], axis=1, inplace=True)

#dataset_all.drop(["name", "kickoff_time", "kickoff_time_formatted"], axis=1, inplace=True)

#dataset_test_all.drop(["name", "kickoff_time", "kickoff_time_formatted"], axis=1, inplace=True)
         
# Remove bad instances located near bottom of set
dataset=dataset.iloc[:67936,:]

# Remove all players with minutes = 0
dataset = dataset[dataset.minutes!=0]

# c=dataset_test.columns
# print(c[46])

# Set the target y as goal_scored and x as remaining columns
dataset_test.drop(["goals_scored"], axis=1, inplace=True)
X = dataset.iloc[:,dataset.columns != "goals_scored"].values
y = dataset.iloc[:,dataset.columns == "goals_scored"].values

SyntaxError: invalid character in identifier (<ipython-input-2-b3b8399c2243>, line 14)

# Split for train(.70) and test(.30) data

In [ ]:
# Splitting the data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=5)

# Lets examine the data before setting it to the different models selected

In [ ]:
total_goals = np.unique(y_train, return_counts=True)

print(total_goals[1])
plt.bar(total_goals[0],total_goals[1])

 # Fitting the data to a Multiple Linear Regression Model Without normalization

In [ ]:
# Fitting the data to a Multiple Linear Regression Model

# Fitting the multiple linear regresion to the training set
mlr_obj_no_norm = LinearRegression()
mlr_obj_no_norm.fit(X_train, y_train)

num_culmn_raw  = X_train[0,:]

print("The weight parameters are:")

for idx, col_name in enumerate(dataset_test.columns[:len(num_culmn_raw)]):
    print("({}) {}: {}".format(idx, col_name, mlr_obj_no_norm.coef_[0][idx]).sorted())

# Predicting on the test set
mlr_y_pred_no_norm = mlr_obj_no_norm.predict(X_test)


# Calculate the Mean Squared Error for the model

In [ ]:
print("Mean Squared Error for MLR:", metrics.mean_squared_error(y_test, mlr_y_pred_no_norm))

# Normalizing the data to make sure no one variable pulls to much on the prediction. (Do We Need to Normalize)

In [ ]:
#Normalizing the features
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Fitting the data to a Multiple Linear Regression Model

In [ ]:
# Fitting the multiple linear regresion to the training set
mlr_obj = LinearRegression()
mlr_obj.fit(X_train, y_train)


num_culmn_raw  = X_train[0,:]

print("The weight parameters are:")

for idx, col_name in enumerate(dataset_test.columns[:len(num_culmn_raw)]):
    print("({}) {}: {}".format(idx, col_name, mlr_obj.coef_[0][idx]))

# Predicting on the test set
mlr_y_pred = mlr_obj.predict(X_test)


# Calculate the Mean Squared Error for the model

In [ ]:
print("Mean Squared Error for MLR:", metrics.mean_squared_error(y_test, mlr_y_pred))

# Using backwards elimination to trim the data

In [ ]:
def backwardElimination(x, sl):
    
    numVars = len(x[0])
    for i in range(0, numVars):
        
        #print("check:", i)

        obj_OLS = sm.OLS(endog = y, exog = x).fit()
        maxVar = max(obj_OLS.pvalues).astype(float)
        #print(maxVar)

        if maxVar > sl:
            
            for j in range(0, numVars - i):
                if (obj_OLS.pvalues[j].astype(float) == maxVar):
                    print(j)
                    x = np.delete(x, j, 1)
                    dataset_test.drop([dataset_test.columns[j]], axis=1, inplace=True)
                    
                    
    obj_OLS.summary()
    return x


X = X.astype(float)
SL = 0.05
X_sig = X[:,:]
X_Modeled = backwardElimination(X_sig, SL)

# Now run the trimed data with a linear regression model

In [ ]:
# Split the data
X_sig_train, X_sig_test, y_sig_train, y_sig_test = train_test_split(
    X_Modeled, y, test_size = 0.3)

mlr_bw_obj = LinearRegression()
mlr_bw_obj.fit(X_sig_train, y_sig_train)

y_sig_pred = mlr_bw_obj.predict(X_sig_test)

num_culmn_sig  = X_Modeled[0,:]

print("The weight parameters are:")

for idx, col_name in enumerate(dataset_test.columns[:len(num_culmn_sig)]):
    print("({}) {}: {}".format(idx, col_name, mlr_bw_obj.coef_[0][idx]))

#print("Mean Squared Error for MLR and Backward Elimination:", metrics.mean_squared_error(y_sig_test, y_sig_pred))

# MLR and Backward Elimination Mean Squared Error

In [ ]:
print("Mean Squared Error for MLR and Backward Elimination:", metrics.mean_squared_error(y_sig_test, y_sig_pred))

# Examine a Decison Tree regresion

In [ ]:
dtr_obj = DecisionTreeRegressor()
dtr_obj.fit(X_train, y_train)

dtr_y_pred = dtr_obj.predict(X_test)

print("Mean Squared Error for Decision Tree:", metrics.mean_squared_error(y_test, dtr_y_pred))

# Examine a Random Forrest model

In [ ]:
num_tree = []
msqe = []

for n in range (10, 301, 10):
    
    num_tree.append(n)
    
    randf_obj = RandomForestRegressor(n_estimators=n)
    randf_obj.fit(X_train, y_train)
    
    randf_y_pred = randf_obj.predict(X_test)
    
    mean_sqr_eror = metrics.mean_squared_error(y_test, 
                                               randf_y_pred)
    
    # print(mean_sqr_eror)
    
    msqe.append(mean_sqr_eror)
    
print('Lowest testing error =', min(msqe))
print('Associated number of trees (n_estimator) =', 
      num_tree[msqe.index(min(msqe))])

In [ ]:
print('Lowest testing error =', min(msqe))
print('Associated number of trees (n_estimator) =', 
      num_tree[msqe.index(min(msqe))])

# PCA model examination

In [ ]:
num_tree_pca = []
pca_error = []

for n in range(1, len(num_culmn_raw)):
    num_tree_pca.append(n)
    
    pca_obj = PCA(n_components=n)
    X_PCA_train = pca_obj.fit_transform(X_train)
    X_PCA_test = pca_obj.transform(X_test)
    components_variance = pca_obj.explained_variance_ratio_

    # Fit a linear regression to the training set
    mlr_pca_obj = LinearRegression()
    mlr_pca_obj.fit(X_PCA_train, y_train)

    y_mlr_pca_pred = mlr_pca_obj.predict(X_PCA_test)
    
    pca_error.append(metrics.mean_squared_error(y_test, y_mlr_pca_pred))
    
    #print(n)
    #print(metrics.mean_squared_error(y_test, y_mlr_pca_pred))
    
print('Lowest testing error =', min(pca_error))
print('Associated number of components (n_components) =', 
      num_tree_pca[pca_error.index(min(pca_error))])

# K-Fold Tests

In [ ]:
# Multiple Linear Regression Raw
modelAccuracies = cross_val_score(estimator=mlr_obj, X=X_train, y=y_train, cv=10)
print("MLR Mean:", modelAccuracies.mean())
print("MLR Standard Deviation:", modelAccuracies.std())

print()

# Multilinear Regression with Backward Elimination
modelAccuracies = cross_val_score(estimator=mlr_bw_obj, X=X_train, y=y_train, cv=10)
print("MLR/Backward Elimination Mean:", modelAccuracies.mean())
print("MLR/Backward Elimination Deviation:", modelAccuracies.std())

print()

# Decision Tree Model
modelAccuracies = cross_val_score(estimator=dtr_obj, X=X_train, y=y_train, cv=10)
print("Decission Tree Mean:", modelAccuracies.mean())
print("Decission Tree Deviation:", modelAccuracies.std())

print()

# Random Forest Model
randf_obj = RandomForestRegressor(n_estimators=num_tree[msqe.index(min(msqe))])
modelAccuracies = cross_val_score(estimator=randf_obj, X=X_train, y=y_train, cv=10)
print("Random Forest Mean:", modelAccuracies.mean())
print("Random Forest Standard Deviation:", modelAccuracies.std())

print()

# PCA Model
modelAccuracies = cross_val_score(estimator=mlr_pca_obj, X=X_train, y=y_train, cv=5)
print("MLR/PCA Mean:", modelAccuracies.mean())
print("MLR/PCA Standard Deviation:", modelAccuracies.std())